# Daily Update

In [ ]:
# hide output
%%capture output

!pip install setuptools
!pip install -U polygon-api-client

In [ ]:
import os
import numpy as np
import pandas as pd
import requests
import json
import pytz
from polygon import RESTClient
from google.colab import drive
from datetime import datetime
from tqdm import tqdm



# Access drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Sentimental/'

# get stock list
stocks = pd.read_csv(os.path.join(path, 'data', 'stock_list.csv'), index_col = 0)
stock_lst = [[company, stocks.loc[company, 'stock_code']] for company in stocks.index]
stocks

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,stock_code
apple,AAPL


## Daily Updates

### methods

In [ ]:
with open(os.path.join(path, 'data', 'api_key_yahoo.txt'), 'r') as file:
    api_key_yahoo = file.read()

with open(os.path.join(path, 'data', 'api_key_eodhd.txt'), 'r') as file:
    api_key_eodhd = file.read()

with open(os.path.join(path, 'data', 'api_key_gpt.txt'), 'r') as file:
    api_key_gpt = file.read()



def get_vwap(stock, date):
    client = RESTClient(api_key_yahoo)

    date = pd.Timestamp(date)
    start_date = date - pd.Timedelta(7, unit='D')
    end_date = pd.Timestamp('01/01/2030')

    data = client.get_aggs(stock[:4], 1, 'day', start_date, end_date)
    times = pd.DataFrame([x.timestamp for x in data], columns = ['time'])
    trading_dates = pd.DatetimeIndex(pd.to_datetime(times['time'], unit='ms').dt.date)

    for i in range(len(trading_dates)):
        if trading_dates[i] > date:
            break

    if i == -1:
        return -1
    else:
        return data[i].vwap


def get_news(stock, date):
    url = "https://eodhd.com/api/news?s={stock}&offset=0&limit=50&api_token={api_key}"
    data = requests.get(url.format(stock=stock, api_key=api_key_eodhd)).json()
    func = lambda date: datetime.fromisoformat(date.replace('Z', '+00:00'))
    return [[stock, func(x['date']), x['title'], x['content'], x['link']] for x in data if x['date'][:10] == date]


def get_response(company, stock, news, score_min = -5, score_max = 5):
    stock += '.US'

    headers = {
        'Authorization': f'Bearer {api_key_gpt}',
        'Content-Type': 'application/json',
    }

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant and setimental analysis master"},
            {"role": "user", "content": "Analyze this finicial news responses to" + company + " with stock code: " + stock + ". Determine the overall sentiment and then just give me score " + str(score_min) + " to " + str(score_max) + " first, and one sentense resoning :" + news}
        ]
    }

    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return "Error:", response.status_code, response.text


def parse(text):
    score = ''

    f = 0
    for i in range(len(text)):
        if f == 0 and i > 1 and text[i-2:i] == ': ':
            f = 1
        elif text[i] == '\n':
            break

        if f == 1:
            score += text[i]

    if score == '':
        return 100
    else:
        return int(score)

### daily update

In [ ]:
# create dataframe
df = pd.DataFrame(columns = ['date', 'company', 'stock', 'datetime', 'title', 'news', 'link', 'gpt_response', 'score', 'vwap'])
df.to_csv(os.path.join(path, 'data', 'records.csv'), index = False)

In [ ]:
def daily_update(year = 0, month = 0, day = 0):
    records_path = os.path.join(path, 'data', 'records.csv')

    if year == 0:
        date = datetime.now(pytz.timezone('America/New_York')).strftime('%Y-%m-%d')
    else:
        year = str(year)
        month = str(month)
        day = str(day)
        if len(month) == 1:
            month = '0' + month
        if len(day) == 1:
            day = '0' + day
        date = '{y}-{m}-{d}'.format(y=year, m=month, d=day)

    ret = []

    df = pd.read_csv(records_path)
    if date in df.date.values:
        print('Date {d} is already processed.'.format(d=date))
        return


    for company, stock in stock_lst:
        print('processing data for {comp} on {d}...'.format(comp=company, d=date))
        data_vwap = get_vwap(stock, date)
        data_news = get_news(stock, date)

        for news in tqdm(data_news):
            row = [date, company]

            response = get_response(company, stock, news[3])
            score = parse(response)

            row += news + [response, score, data_vwap]

            df.loc[len(df)] = row
        print('\n')

    df.to_csv(records_path, index = False)
    return df

In [ ]:
daily_update()

processing data for apple on 2025-04-16...


100%|██████████| 21/21 [00:27<00:00,  1.30s/it]

,date,company,stock,datetime,title,news,link,gpt_response,score,vwap
0,2025-04-16,apple,AAPL,2025-04-16 22:14:03+00:00,"Meta Saw TikTok as ‘Highly Urgent’ Threat, Zuc...","(Bloomberg) -- Mark Zuckerberg, the first witn...",https://finance.yahoo.com/news/meta-saw-tiktok...,Sentiment Score: -3\n\nReasoning: The news art...,-3,195.8892
1,2025-04-16,apple,AAPL,2025-04-16 21:08:46+00:00,Why Apple Inc. (AAPL) Is One of the Best Blue ...,We recently published a list of 15 Best Blue C...,https://finance.yahoo.com/news/why-apple-inc-a...,Sentiment Score: -3\n\nReasoning: The financia...,-3,195.8892
2,2025-04-16,apple,AAPL,2025-04-16 21:03:32+00:00,Trading Day: When the chips are down,"By Jamie McGeever\n\nORLANDO, Florida (Reuters...",https://finance.yahoo.com/news/trading-day-chi...,Sentiment Score: -3\n\nReasoning: The news art...,-3,195.8892
3,2025-04-16,apple,AAPL,2025-04-16 19:21:00+00:00,Big Tech’s China Risks Go Far Beyond Nvidia,"Within the largest companies in the sector, th...",https://finance.yahoo.com/m/2692b5d6-c2b2-3433...,Sentiment Score: 3\n\nReasoning: The overall s...,3,195.8892
4,2025-04-16,apple,AAPL,2025-04-16 19:21:00+00:00,China Risks for Big Tech Go Far Beyond Nvidia,"Within the largest companies in the sector, th...",https://finance.yahoo.com/m/2692b5d6-c2b2-3433...,Sentiment Score: 3\n\nReasoning: The statement...,3,195.8892
5,2025-04-16,apple,AAPL,2025-04-16 16:05:35+00:00,How to invest in Nvidia successfully during Tr...,Listen and subscribe to Opening Bid on Apple P...,https://finance.yahoo.com/news/how-to-invest-i...,Sentiment Score: 2\n\nReasoning: The overall s...,2,195.8892
6,2025-04-16,apple,AAPL,2025-04-16 16:00:03+00:00,Apple market cap falls back below $3 trillion ...,Apple (AAPL) stock fell more than 2.5% on Wedn...,https://finance.yahoo.com/news/apple-market-ca...,Overall sentiment score: -3\n\nReasoning: The ...,-3,195.8892
7,2025-04-16,apple,AAPL,2025-04-16 15:26:09+00:00,Is Apple Inc. (NASDAQ:AAPL) the Best NASDAQ St...,We recently published a list of 13 Best NASDAQ...,https://finance.yahoo.com/news/apple-inc-nasda...,Sentiment Score: -3\n\nReasoning: The financia...,-3,195.8892
8,2025-04-16,apple,AAPL,2025-04-16 15:13:42+00:00,Magnificent Seven Stocks Sell Off: Nvidia Plun...,"Dubbed the Magnificent Seven stocks, Apple, Mi...",https://finance.yahoo.com/m/4205eaa9-f620-3a0b...,Overall sentiment score: -3\n\nReasoning: The ...,-3,195.8892
9,2025-04-16,apple,AAPL,2025-04-16 15:00:53+00:00,Trump's trade war unlikely to bring tech manuf...,President Trump is determined to bring tech ma...,https://finance.yahoo.com/news/trumps-trade-wa...,Sentiment Score: -3\n\nReasoning: The financia...,-3,195.8892


In [ ]:
df = pd.read_csv(os.path.join(path, 'data', 'records.csv'))
df

,date,company,stock,datetime,title,news,link,gpt_response,score,vwap
0,2025-04-16,apple,AAPL,2025-04-16 22:14:03+00:00,"Meta Saw TikTok as ‘Highly Urgent’ Threat, Zuc...","(Bloomberg) -- Mark Zuckerberg, the first witn...",https://finance.yahoo.com/news/meta-saw-tiktok...,Sentiment Score: -3\n\nReasoning: The news art...,-3,195.8892
1,2025-04-16,apple,AAPL,2025-04-16 21:08:46+00:00,Why Apple Inc. (AAPL) Is One of the Best Blue ...,We recently published a list of 15 Best Blue C...,https://finance.yahoo.com/news/why-apple-inc-a...,Sentiment Score: -3\n\nReasoning: The financia...,-3,195.8892
2,2025-04-16,apple,AAPL,2025-04-16 21:03:32+00:00,Trading Day: When the chips are down,"By Jamie McGeever\n\nORLANDO, Florida (Reuters...",https://finance.yahoo.com/news/trading-day-chi...,Sentiment Score: -3\n\nReasoning: The news art...,-3,195.8892
3,2025-04-16,apple,AAPL,2025-04-16 19:21:00+00:00,Big Tech’s China Risks Go Far Beyond Nvidia,"Within the largest companies in the sector, th...",https://finance.yahoo.com/m/2692b5d6-c2b2-3433...,Sentiment Score: 3\n\nReasoning: The overall s...,3,195.8892
4,2025-04-16,apple,AAPL,2025-04-16 19:21:00+00:00,China Risks for Big Tech Go Far Beyond Nvidia,"Within the largest companies in the sector, th...",https://finance.yahoo.com/m/2692b5d6-c2b2-3433...,Sentiment Score: 3\n\nReasoning: The statement...,3,195.8892
5,2025-04-16,apple,AAPL,2025-04-16 16:05:35+00:00,How to invest in Nvidia successfully during Tr...,Listen and subscribe to Opening Bid on Apple P...,https://finance.yahoo.com/news/how-to-invest-i...,Sentiment Score: 2\n\nReasoning: The overall s...,2,195.8892
6,2025-04-16,apple,AAPL,2025-04-16 16:00:03+00:00,Apple market cap falls back below $3 trillion ...,Apple (AAPL) stock fell more than 2.5% on Wedn...,https://finance.yahoo.com/news/apple-market-ca...,Overall sentiment score: -3\n\nReasoning: The ...,-3,195.8892
7,2025-04-16,apple,AAPL,2025-04-16 15:26:09+00:00,Is Apple Inc. (NASDAQ:AAPL) the Best NASDAQ St...,We recently published a list of 13 Best NASDAQ...,https://finance.yahoo.com/news/apple-inc-nasda...,Sentiment Score: -3\n\nReasoning: The financia...,-3,195.8892
8,2025-04-16,apple,AAPL,2025-04-16 15:13:42+00:00,Magnificent Seven Stocks Sell Off: Nvidia Plun...,"Dubbed the Magnificent Seven stocks, Apple, Mi...",https://finance.yahoo.com/m/4205eaa9-f620-3a0b...,Overall sentiment score: -3\n\nReasoning: The ...,-3,195.8892
9,2025-04-16,apple,AAPL,2025-04-16 15:00:53+00:00,Trump's trade war unlikely to bring tech manuf...,President Trump is determined to bring tech ma...,https://finance.yahoo.com/news/trumps-trade-wa...,Sentiment Score: -3\n\nReasoning: The financia...,-3,195.8892


### missing date update

In [ ]:
#daily_update(2024,3,1)